In [1]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from scipy.special import softmax
from detector import SpamMessageDetector

In [2]:
#model_name = "mshenoda/roberta-spam"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForSequenceClassification.from_pretrained(model_name)

spam_detector = SpamMessageDetector("mshenoda/roberta-spam")

text = "<FIRST_NAME_1>, here are our recommendations based on what you viewed - \t?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? Not rendering correctly or images not showing? View in a webpage <URL_1> Jobs you might be interested in <URL_2> And if you need a little help putting together the perfect application, check out our various tips and templates ? (German French only) This email was intended for <NAME_1> <EMAIL_1>. You are receiving our recommendations by email based on what you have been browsing on JobScout24 and on your communication preferences. You can change or stop receiving these emails by opting out in your application profile. Unsubscribe from this e-mail | Help | Privacy Policy Copyright \u00a9 2023 | JobCloud AG, JobScout24.ch, <ADDRESS_1>"
print(f"Is spam: {spam_detector.detect(text)}")

Is spam: 1


In [3]:
import json

with open("email_data_set.json",encoding="utf-8") as f:
    data = json.load(f)

print(data)

[{'id': 1, 'email_subject': 'Kontakt Info', 'sender_name': '<NAME_1>', 'sender_email_address': '<EMAIL_1>', 'recipient_name': 'DATASPORT-Info', 'recipient_email': 'info@datasport.com', 'billing_category': None, 'priority': 'Normal', 'conversation': [{'preferred_reaction': 'forward_to_datasport', 'category': 'request', 'direction': 'incoming', 'body': "Veranstaltung: Davos X Trails Vorname: <FIRST_NAME_1> Nachname: <LAST_NAME_1> Geburtsdatum: <BIRTH_DATE_1> Adresse: <ADDRESS_1> E-Mail: <EMAIL_1> Telefon: <PHONE_NUMBER_1> Mitteilung: Hello there is a problem with my data sport account. I believe you still have an old email address linked to it (<EMAIL_1>). This email address is no longer valid. I have not received confirmation of my number/ entry for Saturday's race (Davos X trails) nor have I received information/ code for the Swiss runners ticket. Please can you contact me urgently to resolve this. Thank you. Best regards, <FRIST_NAME_2> IP: <IP_ADDRESS_1>"}, {'preferred_reaction': Non

In [4]:
true_labels = []
predicted_labels = []


for item in data:
    for conversation in item['conversation']:
        text = conversation['body']
        category = conversation.get('category', 'not spam')
        
        true_label = 1 if category == 'spam' else 0
        prediction = spam_detector.detect(text)
        
        if true_label != prediction:
            print(true_label, prediction,text)
        
        predicted_label = 1 if prediction == 'spam' else 0
        true_labels.append(true_label)
        predicted_labels.append(predicted_label)

correct_predictions = sum(1 for true, pred in zip(true_labels, predicted_labels) if true == pred)
accuracy = correct_predictions / len(true_labels)

print(f'Accuracy: {accuracy * 100:.2f}%')

0 1 Dear <FIRST_NAME_3> Thank you for the quick reply. Can you please also look in to the email address associated with my account? Please confirm that <EMAIL_1> is the ONLY email address connected to the account and that all notifications/ registration confirmation etc are sent to this email. Best regards, <FIRST_NAME_2> ________________________________________ <NAME_1> • Solicitor • <PHONE_NUMBER_1> <EMAIL_1> • www.rethinklegal.ch
0 1 Hello <NAME_1>, thank you for registering. <URL_1> Registration confirmation TORTOUR 15th Edition 2023 <DATE_1> Hello <NAME_1> Wow, you have registered several people at once. Thank you very much! We are happy to inform you that we have received your registration, so there is nothing standing in the way of your participation. We wish you all successful training sessions and lots of fun at the event. Your Datasport team Registration details Registration date: <DATE_1 Registration status: CONFIRMED Payment status: PAID Team <FIRST_NAME_1> & <FIRST_NAME_2>